In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens100k/movies.csv
/kaggle/input/movielens100k/ratings.csv
/kaggle/input/movielens100k/u.data
/kaggle/input/movielens100k/tags.csv
/kaggle/input/movielens100k/links.csv


In [2]:
movies = pd.read_csv("/kaggle/input/movielens100k/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv("/kaggle/input/movielens100k/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
tags = pd.read_csv("/kaggle/input/movielens100k/tags.csv")
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [5]:
links = pd.read_csv("/kaggle/input/movielens100k/links.csv")
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
# Return reshaped DataFrame organized by given index / column values.
user_item_m = ratings.pivot('userId','movieId','rating').fillna(0)
print(f'Shape: {user_item_m.shape}')

Shape: (671, 9066)


In [8]:
user_item_m.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine similarity between users
X_user = cosine_similarity(user_item_m) # user similarity matrix

In [10]:
print(X_user.shape)

(671, 671)


In [11]:
print(X_user)

[[1.         0.         0.         ... 0.06291708 0.         0.01746565]
 [0.         1.         0.12429498 ... 0.02413984 0.17059464 0.1131753 ]
 [0.         0.12429498 1.         ... 0.08098382 0.13660585 0.17019275]
 ...
 [0.06291708 0.02413984 0.08098382 ... 1.         0.04260878 0.08520194]
 [0.         0.17059464 0.13660585 ... 0.04260878 1.         0.22867673]
 [0.01746565 0.1131753  0.17019275 ... 0.08520194 0.22867673 1.        ]]


In [35]:
def movie_recommender(user_item_m, X_user, user, k=20, top_n=10):
    user_ix = user_item_m.index.get_loc(user)
    print(user_ix)
    user_similarities = X_user[user_ix]
    print(user_similarities)
    most_similar_users = user_item_m.index[user_similarities.argpartition(-k)[-k:]]
    
    rec_movies = user_item_m.loc[most_similar_users].mean(0).sort_values(ascending=False)
    m_seen_movies = user_item_m.loc[user].gt(0)
    seen_movies = m_seen_movies.index[m_seen_movies].tolist()
    rec_movies = rec_movies.drop(seen_movies).head(top_n)
    return rec_movies.index.to_frame().reset_index(drop=True).merge(movies)

In [36]:
recommendation = movie_recommender(user_item_m, X_user, user = 2)

1
[0.         1.         0.12429498 0.11882103 0.10364614 0.
 0.21298521 0.11319045 0.11333307 0.04321284 0.04961008 0.01607144
 0.12896041 0.         0.13658035 0.0484117  0.06042607 0.05476582
 0.2564788  0.16795476 0.18085948 0.14379021 0.12270409 0.21479948
 0.         0.11196036 0.10747578 0.01643669 0.         0.16644782
 0.057055   0.45254604 0.00472652 0.10954598 0.04219155 0.35512971
 0.02034247 0.1032038  0.24618017 0.01932264 0.06489199 0.16785178
 0.14510517 0.04598977 0.         0.         0.37746843 0.04475632
 0.04308694 0.48065198 0.         0.01333505 0.01798432 0.
 0.         0.07153368 0.11424459 0.04670051 0.17868909 0.03128211
 0.11451897 0.         0.08556895 0.39739867 0.         0.12218073
 0.3042287  0.15657772 0.12932312 0.05470244 0.02310209 0.08944138
 0.12124157 0.14425409 0.09201019 0.         0.17646219 0.14044616
 0.1200188  0.07217386 0.00753448 0.49407109 0.09714676 0.04425643
 0.43227421 0.39402716 0.02498231 0.21830408 0.14013553 0.01416118
 0.062794

In [37]:
print(recommendation)

   movieId                              title  \
0      318   Shawshank Redemption, The (1994)   
1      380                   True Lies (1994)   
2      595        Beauty and the Beast (1991)   
3      316                    Stargate (1994)   
4      440                        Dave (1993)   
5      597                Pretty Woman (1990)   
6       34                        Babe (1995)   
7      344  Ace Ventura: Pet Detective (1994)   
8      329      Star Trek: Generations (1994)   
9      434                 Cliffhanger (1993)   

                                            genres  
0                                      Crime|Drama  
1         Action|Adventure|Comedy|Romance|Thriller  
2  Animation|Children|Fantasy|Musical|Romance|IMAX  
3                          Action|Adventure|Sci-Fi  
4                                   Comedy|Romance  
5                                   Comedy|Romance  
6                                   Children|Drama  
7                                   